#### Package 설치

In [ ]:
#!pip install docx2txt
#!pip install langchain-upstage

In [8]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

24


#### 1. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [9]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('../data/tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

print(len(document_list))
print(type(document_list[0]))
print(document_list[:2])

225
<class 'langchain_core.documents.base.Document'>
[Document(metadata={'source': '../data/tax_with_table.docx'}, page_content='소득세법\n\n소득세법\n\n[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(재산세제과(양도소득세)) 044-215-4314\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236\n\n\n\n\t제1장 총칙 <개정 2009. 12. 31.>\t\n\n\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n\n[본조신설 2009. 12. 31.]\n\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무

In [10]:
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
print(embeddings)

client=<openai.resources.embeddings.Embeddings object at 0x12dd8fe00> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x12f168830> model='solar-embedding-1-large' dimensions=None upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1/solar' embedding_ctx_length=4096 embed_batch_size=10 allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers={'x-upstage-client': 'langchain'} default_query=None http_client=None http_async_client=None


In [11]:
from langchain_community.vectorstores import FAISS

# 데이터를 처음 저장할 때 
database = FAISS.from_documents(documents=document_list, embedding=embeddings)
# 로컬 파일로 저장
database.save_local("faiss_docdb")

# 이미 저장된 데이터를 사용할 때 
# database = FAISS.load_local("faiss_docdb", embedding, allow_dangerous_deserialization=True)
print(database)

#### 2. 답변 생성을 위한 Retrieval

- `FAISS`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [12]:
query = '비과세소득에는 어떤것들이 있나요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=6)

print(len(retrieved_docs))
print(type(retrieved_docs[0]))
print(retrieved_docs[0].metadata)

6
<class 'langchain_core.documents.base.Document'>
{'source': '../data/tax_with_table.docx'}


In [13]:
print(retrieved_docs[0].page_content[:100])

마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품

바. 「문화재보호법」에 따라 국가지정문화재로 지정된 서화ㆍ골동품의 양도로 발생


#### 3. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [14]:

from langchain_upstage import ChatUpstage

print("===> 6. 생성 → LLM으로 답변 생성")
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

===> 6. 생성 → LLM으로 답변 생성
client=<openai.resources.chat.completions.completions.Completions object at 0x12f517380> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12f5e0830> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


In [15]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


#### 4. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [16]:
from langchain.chains import RetrievalQA
from pprint import pprint

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [17]:
query = '비과세소득에는 어떤것들이 있나요?'

ai_message = qa_chain.invoke({"query": query})
pprint(ai_message)

{'query': '비과세소득에는 어떤것들이 있나요?',
 'result': '비과세소득에는 국군포로의 위로지원금, 국가지정문화재 서화·골동품 양도 소득, 박물관·미술관 양도 소득, 종교인소득 중 '
           '학자금·식사대·실비변상적 지급액·월 20만원 이내 보육지원금·사택 이익, 무보수 위원회 수당 등이 포함됩니다. (출처: '
           '제공된 법조문 제12조 및 관련 항목)  \n'
           '\n'
           '답변은 주어진 법조문 내 명시된 내용을 기반으로 작성되었습니다.'}


In [18]:
query = '과세소득의 범위 및 소득의 구분에는 어떤것들이 있나요?'

ai_message = qa_chain.invoke({"query": query})
pprint(ai_message)

{'query': '과세소득의 범위 및 소득의 구분에는 어떤것들이 있나요?',
 'result': '과세소득의 범위는 거주자의 경우 모든 소득(제3조①), 비거주자는 국내원천소득(제3조②)에 대해 과세됩니다. 소득 '
           '구분은 거주자의 경우 종합소득(이자, 배당, 사업, 근로, 연금, 기타소득), 퇴직소득, 금융투자소득, 양도소득으로 '
           '분류되며(제4조①), 비거주자는 국내원천소득에 따라 구분됩니다(제4조③). 신탁소득에 대해서는 수익자 또는 위탁자에게 '
           '귀속될 수 있습니다(제2조의3).'}


* LangChain 기반의 RAG(Retrieval-Augmented Generation) 파이프라인을 구현하여 DOCX 문서를 로드, 임베딩, 검색, 그리고 LLM을 통한 답변 생성

In [19]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.document_loaders import Docx2txtLoader
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # 특정 경고 유형만 무시

#  1. 환경 변수 로드
load_dotenv()

#  2. OpenAI API 키 확인
OPENAI_API_KEY = os.getenv("UPSTAGE_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("UPSTAGE API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

#  3. DOCX 파일 로드 및 텍스트 추출 (Docx2txtLoader 활용)
def load_docx(file_path):
    """DOCX 파일에서 텍스트를 추출하는 함수."""
    try:
        loader = Docx2txtLoader(file_path)
        documents = loader.load()
        text = "\n".join([doc.page_content for doc in documents])
        if not text.strip():
            raise ValueError("문서에서 텍스트를 추출할 수 없습니다.")
        return text
    except Exception as e:
        raise RuntimeError(f"문서 로딩 실패: {str(e)}")

#  4. 문서 분할 함수
def split_text(text, chunk_size=500, chunk_overlap=50):
    """텍스트를 지정된 크기의 청크로 분할하는 함수."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return splitter.split_text(text)

#  5. 벡터 데이터베이스(FAISS) 생성 함수
def create_vector_store(text_chunks, embedding_model):
    """텍스트 청크를 임베딩하고 FAISS 벡터 저장소에 저장."""
    try:
        #List[Document]
        documents = [Document(page_content=chunk) for chunk in text_chunks]
        vector_store = FAISS.from_documents(documents, embedding_model)
        vector_store.save_local("faiss_docdb")
        return vector_store
    except Exception as e:
        raise RuntimeError(f"벡터 저장소 생성 실패: {str(e)}")

#  6. LLM을 활용한 질문 응답 함수
def query_with_llm(query, vector_store):
    """LLM을 사용하여 검색된 문서 기반으로 답변 생성."""
    try:
        # LLM 모델 설정
        llm = ChatUpstage(
            model="solar-pro",
            base_url="https://api.upstage.ai/v1",
            temperature=0.5
        )
        
        # 프롬프트 로드 (RAG 최적화된 LangChain Hub 프롬프트 사용)
        prompt = hub.pull("rlm/rag-prompt")

        # RetrievalQA 체인 생성
        qa_chain = RetrievalQA.from_chain_type(
            llm, 
            retriever=vector_store.as_retriever(),
            chain_type_kwargs={"prompt": prompt}
        )

        # LLM 응답 생성
        ai_message = qa_chain.invoke({"query": query})
        print(ai_message)
        return ai_message["result"]

    except Exception as e:
        raise RuntimeError(f"LLM 응답 생성 실패: {str(e)}")

#  실행 예제
if __name__ == "__main__":
    # DOCX 파일 경로
    docx_path = "../data/tax_with_table.docx"
    
    # 1. 문서 로드
    text = load_docx(docx_path)
    print("문서 로드 완료")
    
    # 2. 문서 분할
    text_chunks = split_text(text)
    print(f"문서 분할 완료: {len(text_chunks)}개 청크 생성")
    
    # 3. 임베딩 모델 초기화
    embedding_model = UpstageEmbeddings(model="solar-embedding-1-large")
    
    # 4. 벡터 저장소 생성
    vector_store = create_vector_store(text_chunks, embedding_model)
    print("벡터 저장소 생성 완료")
    
    # 5. 질의 실행
    query = "총수입금액 불산입에 대하여 설명해 주세요."
    results = query_with_llm(query, vector_store)
    
    # 6. AI 응답 출력
    print("\n AI의 답변:")
    print(results)

문서 로드 완료
문서 분할 완료: 720개 청크 생성
벡터 저장소 생성 완료
{'query': '총수입금액 불산입에 대하여 설명해 주세요.', 'result': '총수입금액 불산입은 소득세 계산 시 특정 항목을 총수입금액에 포함하지 않는 것을 의미합니다. 이는 개별소비세·주세(원자재 등 제외), 국세·지방세 환급가산금, 부가가치세 매출세액, 석유판매업자 환급세액 등이 해당됩니다. 단, 무상 자산 가액 중 이월결손금 보전 충당액도 제외됩니다. (참고: 제26조 및 관련 조항)'}

 AI의 답변:
총수입금액 불산입은 소득세 계산 시 특정 항목을 총수입금액에 포함하지 않는 것을 의미합니다. 이는 개별소비세·주세(원자재 등 제외), 국세·지방세 환급가산금, 부가가치세 매출세액, 석유판매업자 환급세액 등이 해당됩니다. 단, 무상 자산 가액 중 이월결손금 보전 충당액도 제외됩니다. (참고: 제26조 및 관련 조항)


In [20]:
from pprint import pprint

pprint(results)

('총수입금액 불산입은 소득세 계산 시 특정 항목을 총수입금액에 포함하지 않는 것을 의미합니다. 이는 개별소비세·주세(원자재 등 제외), '
 '국세·지방세 환급가산금, 부가가치세 매출세액, 석유판매업자 환급세액 등이 해당됩니다. 단, 무상 자산 가액 중 이월결손금 보전 충당액도 '
 '제외됩니다. (참고: 제26조 및 관련 조항)')


###  개선된 Source Level1
```python
RuntimeError: 벡터 저장소 생성 실패: Error code: 400 - 
{'error': {
    'message': 'Requested 313741 tokens, max 300000 tokens per request', 
    'type': 'max_tokens_per_request', 
    'param': None, 'code': 'max_tokens_per_request'
    }
}
```

In [21]:
import os
from dotenv import load_dotenv

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.document_loaders import Docx2txtLoader
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
import re
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

#  1. 환경 변수 로드
load_dotenv()

#  2. OpenAI API 키 확인
OPENAI_API_KEY = os.getenv("UPSTAGE_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("UPSTAGE API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

# 2. 한국어 법률 문서 전용 텍스트 전처리 함수
def preprocess_korean_legal_text(text):
    """한국어 법률 문서를 위한 전처리."""
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)
    
    # 조항 번호 정규화 (제1조, 제2조 등)
    text = re.sub(r'제(\d+)조', r'제\1조', text)
    
    # 항 번호 정규화
    text = re.sub(r'①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩', 
                  lambda m: f"제{ord(m.group()) - ord('①') + 1}항", text)
    
    # 호 번호 정규화
    text = re.sub(r'(\d+)\.\s', r'제\1호 ', text)
    
    return text.strip()

# 3. 개선된 문서 분할 함수
def advanced_split_text(text, chunk_size=800, chunk_overlap=200):
    """법률 문서에 최적화된 텍스트 분할."""
    # 전처리
    text = preprocess_korean_legal_text(text)
    
    # 법률 문서 구조를 고려한 분할자
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=[
            "\n제", "\n**제",  # 조항 분할
            "\n①", "\n②", "\n③", "\n④", "\n⑤",  # 항 분할
            "\n1.", "\n2.", "\n3.", "\n4.", "\n5.",  # 호 분할
            "\n가.", "\n나.", "\n다.", "\n라.", "\n마.",  # 목 분할
            "\n\n",  # 문단 분할
            "\n",    # 줄 분할
            ". ",    # 문장 분할
            " ",     # 단어 분할
            ""       # 문자 분할
        ]
    )
    return splitter.split_text(text)

# 4. 개선된 문서 로더
def load_docx_advanced(file_path):
    """개선된 DOCX 파일 로더."""
    try:
        loader = Docx2txtLoader(file_path)
        documents = loader.load()
        text = "\n".join([doc.page_content for doc in documents])
        
        if not text.strip():
            raise ValueError("문서에서 텍스트를 추출할 수 없습니다.")
        
        # 기본 정리
        text = re.sub(r'\n\s*\n', '\n\n', text)  # 여러 개의 빈 줄을 두 개로 통일
        text = re.sub(r'[ \t]+', ' ', text)  # 여러 공백을 하나로 통일
        
        return text
    except Exception as e:
        raise RuntimeError(f"문서 로딩 실패: {str(e)}")

# 5. 벡터 저장소 생성 함수
def create_vector_store(text_chunks, embedding_model):
    """메타데이터가 포함된 벡터 저장소 생성."""
    try:
        documents = []
        for i, chunk in enumerate(text_chunks):
            # 메타데이터 추가
            metadata = {
                'chunk_id': i,
                'chunk_length': len(chunk),
                'chunk_type': 'legal_document'
            }
            
            # 조항 정보 추출
            if '제' in chunk and '조' in chunk:
                article_match = re.search(r'제(\d+)조', chunk)
                if article_match:
                    metadata['article'] = f"제{article_match.group(1)}조"
            
            documents.append(Document(page_content=chunk, metadata=metadata))
        # 
        vector_store = FAISS.from_documents(documents, embedding_model)
        vector_store.save_local("faiss_docdb")
        return vector_store, documents
    except Exception as e:
        raise RuntimeError(f"벡터 저장소 생성 실패: {str(e)}")

# 6. 키워드 기반 검색 함수
def keyword_search(query, documents, k=5):
    """간단한 키워드 기반 검색."""
    query_words = set(query.lower().split())
    
    # 각 문서의 점수 계산
    scores = []
    for i, doc in enumerate(documents):
        content_words = set(doc.page_content.lower().split())
        
        # 교집합 단어 수로 점수 계산
        intersection = query_words.intersection(content_words)
        score = len(intersection) / len(query_words) if query_words else 0
        
        # 정확한 구문 매칭 보너스
        if query.lower() in doc.page_content.lower():
            score += 0.5
        
        scores.append((score, i, doc))
    
    # 점수순 정렬하여 상위 k개 반환
    scores.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, _, doc in scores[:k]]

# 7. 하이브리드 검색 함수
def hybrid_search(query, vector_store, documents, k=5, alpha=0.7):
    """벡터 검색과 키워드 검색을 결합한 하이브리드 검색."""
    
    # 1. 벡터 유사도 검색
    vector_results = vector_store.similarity_search(query, k=k*2)  # 더 많이 가져와서 다양성 확보
    
    # 2. 키워드 검색
    keyword_results = keyword_search(query, documents, k=k*2)
    
    # 3. 결과 합치기 및 점수 계산
    combined_results = {}
    
    # 벡터 검색 결과 점수 (alpha 가중치)
    for i, doc in enumerate(vector_results):
        doc_id = doc.page_content
        vector_score = alpha * (1.0 - i / len(vector_results))
        combined_results[doc_id] = {
            'document': doc,
            'score': vector_score,
            'vector_rank': i + 1
        }
    
    # 키워드 검색 결과 점수 ((1-alpha) 가중치)
    for i, doc in enumerate(keyword_results):
        doc_id = doc.page_content
        keyword_score = (1 - alpha) * (1.0 - i / len(keyword_results))
        
        if doc_id in combined_results:
            # 이미 있는 문서면 점수 합산
            combined_results[doc_id]['score'] += keyword_score
            combined_results[doc_id]['keyword_rank'] = i + 1
        else:
            # 새로운 문서면 추가
            combined_results[doc_id] = {
                'document': doc,
                'score': keyword_score,
                'keyword_rank': i + 1
            }
    
    # 4. 점수순으로 정렬하여 상위 k개 반환
    sorted_results = sorted(combined_results.values(), key=lambda x: x['score'], reverse=True)
    return [result['document'] for result in sorted_results[:k]]

# 8. 한국어 법률 문서 전용 프롬프트 생성 함수
def create_korean_legal_prompt():
    """한국어 법률 문서용 프롬프트 템플릿."""
    template = """당신은 한국 세법 전문가입니다. 주어진 법률 문서를 바탕으로 정확하고 자세한 답변을 제공해야 합니다.

다음 규칙을 반드시 따르세요:
1. 법조문의 조항, 항, 호, 목을 정확히 인용하세요
2. 전문 용어를 사용할 때는 쉬운 설명을 함께 제공하세요
3. 관련 조항들 간의 연관성을 설명하세요
4. 실무적 적용 방법도 함께 설명하세요
5. 불확실한 내용이 있으면 명시적으로 언급하세요

참고 문서:
{context}

질문: {question}

위 법률 문서를 바탕으로 정확하고 자세한 답변을 제공해주세요. 관련 조항을 인용하며 설명해주세요."""

    return PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

# 9. 질문 응답 함수
def query_with_llm(query, vector_store, documents):
    """개선된 LLM 기반 질문 응답."""
    try:
        # LLM 모델 설정
        llm = ChatUpstage(
            model="solar-pro",
            base_url="https://api.upstage.ai/v1",
            temperature=0.5
        )
        
        print(f"질의: {query}")
        print("하이브리드 검색 수행 중...")
        
        # 하이브리드 검색으로 관련 문서 찾기
        relevant_docs = hybrid_search(query, vector_store, documents, k=7, alpha=0.7)
        
        print(f"검색된 관련 문서: {len(relevant_docs)}개")
        
        # 컨텍스트 구성
        context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(relevant_docs)])
        
        # 한국어 법률 문서용 프롬프트
        prompt = create_korean_legal_prompt()
        
        # 프롬프트 생성
        formatted_prompt = prompt.format(context=context, question=query)
        
        print("LLM 응답 생성 중...")
        
        # LLM 응답 생성
        response = llm.invoke(formatted_prompt)
        
        return {
            "answer": response.content,
            "source_documents": relevant_docs,
            "context_used": context
        }
        
    except Exception as e:
        raise RuntimeError(f"LLM 응답 생성 실패: {str(e)}")

# 10. 컨텍스트 품질 평가 함수
def evaluate_context_quality(query, retrieved_docs):
    """검색된 문서의 품질을 간단히 평가."""
    query_words = set(query.lower().split())
    
    quality_scores = []
    for doc in retrieved_docs:
        doc_words = set(doc.page_content.lower().split())
        
        # 키워드 매칭 점수
        keyword_match = len(query_words.intersection(doc_words)) / len(query_words)
        
        # 문서 길이 점수 (너무 짧거나 길지 않은 것이 좋음)
        length_score = min(len(doc.page_content) / 1000, 1.0)
        
        # 종합 점수
        total_score = (keyword_match * 0.7) + (length_score * 0.3)
        quality_scores.append(total_score)
    
    avg_quality = sum(quality_scores) / len(quality_scores) if quality_scores else 0
    return avg_quality

# 실행 예제
if __name__ == "__main__":
    # DOCX 파일 경로
    docx_path = "../data/tax_with_table.docx"
    
    print("=== 개선된 RAG 파이프라인 실행 ===\n")
    
    # 1. 문서 로드
    print("1. 문서 로드 중...")
    text = load_docx_advanced(docx_path)
    print(f"   문서 로드 완료: {len(text):,} 문자\n")
    
    # 2. 개선된 문서 분할
    print("2. 문서 분할 중...")
    text_chunks = advanced_split_text(text, chunk_size=800, chunk_overlap=200)
    print(f"   문서 분할 완료: {len(text_chunks)}개 청크 생성\n")
    
    # 3. 임베딩 모델 초기화
    print("3. 임베딩 모델 초기화...")
    embedding_model = UpstageEmbeddings(model="solar-embedding-1-large")
    print("   임베딩 모델 초기화 완료\n")
    
    # 4. 벡터 저장소 생성
    print("4. 벡터 저장소 생성 중...")
    vector_store, documents = create_vector_store(text_chunks, embedding_model)
    print("    벡터 저장소 생성 완료\n")
    
    # 5. 질의 실행
    print("5. 질의 실행 중...")
    query = "총수입금액 불산입에 대하여 설명해 주세요."
    results = query_with_llm(query, vector_store, documents)
    

=== 개선된 RAG 파이프라인 실행 ===

1. 문서 로드 중...
   문서 로드 완료: 289,214 문자

2. 문서 분할 중...
   문서 분할 완료: 511개 청크 생성

3. 임베딩 모델 초기화...
   임베딩 모델 초기화 완료

4. 벡터 저장소 생성 중...
    벡터 저장소 생성 완료

5. 질의 실행 중...
질의: 총수입금액 불산입에 대하여 설명해 주세요.
하이브리드 검색 수행 중...
검색된 관련 문서: 7개
LLM 응답 생성 중...


In [22]:
from pprint import pprint

pprint(results)

{'answer': '### 총수입금액 불산입 개요  \n'
           '총수입금액 불산입은 소득세법에서 특정 금액을 해당 과세기간의 소득 계산 시 총수입금액에 포함하지 않도록 규정한 '
           '제도입니다. 이는 이중과세 방지, 경제적 실질 반영, 또는 정책적 목적에 따라 적용됩니다. 주요 조항은 **소득세법 '
           '제26조(총수입금액 불산입)**와 관련 특례 조항들로 구성됩니다.\n'
           '\n'
           '---\n'
           '\n'
           '### 1. **총수입금액 불산입 대상 및 법적 근거**  \n'
           '#### (1) **세금 환급금 및 충당금**  \n'
           '- **조항**: 제26조제1항  \n'
           '  > "거주자가 소득세 또는 개인지방소득세를 환급받았거나 환급받을 금액 중 다른 세액에 충당한 금액은 해당 과세기간의 '
           '소득금액을 계산할 때 총수입금액에 산입하지 아니한다."  \n'
           '  - **설명**: 세금 환급금은 원래 납부자의 권리이므로 소득으로 간주하지 않습니다. 예를 들어, 연말정산 후 '
           '환급받은 금액은 총수입금액에 포함되지 않습니다.  \n'
           '  - **연관 조항**: 제33조(필요경비 불산입)와 구분 필요. 환급금은 수입에서 제외되지만, 필요경비 불산입은 '
           '지출 비용을 인정하지 않는 것입니다.\n'
           '\n'
           '#### (2) **무상 자산 수령액 및 채무 면제액**  \n'
           '- **조항**: 제26조제2항  \n'
           '  > "거주자가 무상으로 받은 자산의 가액(복식부기의무자가 국가 등으로부터 받은 금액 제외)과 채무 면제액 중 '
           '이월결손금 보전에 충당된 금액은 총수입금액에 산

In [23]:
# 6. 컨텍스트 품질 평가
context_quality = evaluate_context_quality(query, results["source_documents"])

# 7. 결과 출력
print("\n" + "="*60)
print(" AI의 답변:")
print("="*60)
print(results["answer"])

print("\n" + "="*60)
print(" 검색 결과 요약:")
print("="*60)
print(f"• 참고한 문서 조각 수: {len(results['source_documents'])}개")
print(f"• 컨텍스트 품질 점수: {context_quality:.2f}/1.00")
print(f"• 총 컨텍스트 길이: {len(results['context_used']):,} 문자")

print("\n" + "="*60)
print("📄 참고한 문서 미리보기:")
print("="*60)
for i, doc in enumerate(results["source_documents"][:3]):  # 상위 3개만 미리보기
    preview = doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content
    print(f"\n[문서 {i+1}] {preview}")
print("="*60)


 AI의 답변:
### 총수입금액 불산입 개요  
총수입금액 불산입은 소득세법에서 특정 금액을 해당 과세기간의 소득 계산 시 총수입금액에 포함하지 않도록 규정한 제도입니다. 이는 이중과세 방지, 경제적 실질 반영, 또는 정책적 목적에 따라 적용됩니다. 주요 조항은 **소득세법 제26조(총수입금액 불산입)**와 관련 특례 조항들로 구성됩니다.

---

### 1. **총수입금액 불산입 대상 및 법적 근거**  
#### (1) **세금 환급금 및 충당금**  
- **조항**: 제26조제1항  
  > "거주자가 소득세 또는 개인지방소득세를 환급받았거나 환급받을 금액 중 다른 세액에 충당한 금액은 해당 과세기간의 소득금액을 계산할 때 총수입금액에 산입하지 아니한다."  
  - **설명**: 세금 환급금은 원래 납부자의 권리이므로 소득으로 간주하지 않습니다. 예를 들어, 연말정산 후 환급받은 금액은 총수입금액에 포함되지 않습니다.  
  - **연관 조항**: 제33조(필요경비 불산입)와 구분 필요. 환급금은 수입에서 제외되지만, 필요경비 불산입은 지출 비용을 인정하지 않는 것입니다.

#### (2) **무상 자산 수령액 및 채무 면제액**  
- **조항**: 제26조제2항  
  > "거주자가 무상으로 받은 자산의 가액(복식부기의무자가 국가 등으로부터 받은 금액 제외)과 채무 면제액 중 이월결손금 보전에 충당된 금액은 총수입금액에 산입하지 아니한다."  
  - **설명**: 무상으로 받은 자산(예: 증여)은 일반적으로 총수입금액에 산입되지만, **이월결손금 보전**에 사용된 경우 예외입니다.  
  - **실무 적용**: 결손금 보전을 위한 자금 조달로 간주하여 과세 이연 효과가 있습니다.

#### (3) **이월 소득금액**  
- **조항**: 제26조제3항  
  > "이전 과세기간으로부터 이월된 소득금액은 해당 과세기간의 총수입금액에 산입하지 아니한다."  
  - **설명**: 과세되지 않은 소득을 이월받은 경우 중복 과세를 방지하기 위한 

###  개선된 Source Level2
* Prompt 개선

In [24]:
import os
from dotenv import load_dotenv

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.document_loaders import Docx2txtLoader
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

import re
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# 1. 환경 변수 로드 및 API 키 설정
load_dotenv()

#  2. OpenAI API 키 확인
OPENAI_API_KEY = os.getenv("UPSTAGE_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("UPSTAGE API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

# 2. 한국어 법률 문서 전용 텍스트 전처리 함수
def preprocess_korean_legal_text(text):
    """
    한국어 법률 문서의 구조를 고려한 텍스트 전처리 함수
    
    Args:
        text (str): 원본 텍스트
        
    Returns:
        str: 전처리된 텍스트
        
    주요 처리 내용:
        - 불필요한 공백 및 개행 정리
        - 법조문 번호 정규화 (제1조, 제2조 등)
        - 항 번호를 아라비아 숫자로 변환 (①→제1항)
        - 호 번호 정규화
    """
    # 연속된 공백을 하나의 공백으로 통일
    text = re.sub(r'\s+', ' ', text)
    
    # 조항 번호 정규화: "제 1 조" -> "제1조" 형태로 통일
    text = re.sub(r'제(\d+)조', r'제\1조', text)
    
    # 원문자 항 번호를 아라비아 숫자로 변환하여 검색 정확도 향상
    # ①②③④⑤⑥⑦⑧⑨⑩ -> 제1항, 제2항, ... 제10항
    text = re.sub(r'①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩', 
                  lambda m: f"제{ord(m.group()) - ord('①') + 1}항", text)
    
    # 호 번호 정규화: "1. " -> "제1호 " 형태로 변환
    text = re.sub(r'(\d+)\.\s', r'제\1호 ', text)
    
    return text.strip()

# 3. 법률 문서에 최적화된 텍스트 분할 함수
def advanced_split_text(text, chunk_size=600, chunk_overlap=100):
    """
    법률 문서의 구조적 특성을 고려한 지능적 텍스트 분할
    
    Args:
        text (str): 분할할 텍스트
        chunk_size (int): 각 청크의 목표 크기 (문자 수)
        chunk_overlap (int): 청크 간 중복되는 문자 수
        
    Returns:
        list: 분할된 텍스트 청크들의 리스트
        
    특징:
        - 법률 문서의 계층 구조(조>항>호>목)를 고려한 분할 우선순위
        - 의미적 완성도를 유지하면서 분할
        - 토큰 한도를 고려한 적절한 크기 설정
    """
    # 텍스트 전처리 수행
    text = preprocess_korean_legal_text(text)
    
    # 법률 문서 구조를 고려한 분할 구분자들을 우선순위대로 설정
    # 상위 구조부터 하위 구조 순서로 분할을 시도
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=[
            "\n제", "\n**제",  # 조항 단위 분할 (가장 우선)
            "\n①", "\n②", "\n③", "\n④", "\n⑤",  # 항 단위 분할
            "\n1.", "\n2.", "\n3.", "\n4.", "\n5.",  # 호 단위 분할
            "\n가.", "\n나.", "\n다.", "\n라.", "\n마.",  # 목 단위 분할
            "\n\n",  # 문단 단위 분할
            "\n",    # 줄 단위 분할
            ". ",    # 문장 단위 분할
            " ",     # 단어 단위 분할
            ""       # 문자 단위 분할 (최후 수단)
        ]
    )
    return splitter.split_text(text)

# 4. DOCX 파일 로딩 및 전처리 함수
def load_docx_advanced(file_path):
    """
    DOCX 파일을 로드하고 기본적인 텍스트 정리를 수행
    
    Args:
        file_path (str): DOCX 파일 경로
        
    Returns:
        str: 정리된 텍스트
        
    Raises:
        RuntimeError: 파일 로딩 실패 시
        ValueError: 텍스트 추출 실패 시
    """
    try:
        # Docx2txtLoader를 사용하여 DOCX 파일에서 텍스트 추출
        loader = Docx2txtLoader(file_path)
        documents = loader.load()
        text = "\n".join([doc.page_content for doc in documents])
        
        # 텍스트가 비어있는지 확인
        if not text.strip():
            raise ValueError("문서에서 텍스트를 추출할 수 없습니다. 파일이 비어있거나 손상되었을 수 있습니다.")
        
        # 기본적인 텍스트 정리 작업
        # 연속된 빈 줄을 두 개의 줄바꿈으로 통일
        text = re.sub(r'\n\s*\n', '\n\n', text)
        # 연속된 공백과 탭을 하나의 공백으로 통일
        text = re.sub(r'[ \t]+', ' ', text)
        
        return text
    except Exception as e:
        raise RuntimeError(f"문서 로딩 실패: {str(e)}")

# 5. 배치 처리를 통한 벡터 저장소 생성 함수
def create_vector_store(text_chunks, embedding_model, batch_size=30):
    """
    텍스트 청크들을 배치 단위로 처리하여 벡터 저장소 생성
    토큰 한도 초과 문제를 해결하기 위해 배치 처리 방식 적용
    
    Args:
        text_chunks (list): 분할된 텍스트 청크들
        embedding_model: OpenAI 임베딩 모델 객체
        batch_size (int): 한 번에 처리할 청크 수 (기본값: 30)
        
    Returns:
        tuple: (FAISS 벡터 저장소, Document 객체들의 리스트)
        
    처리 과정:
        1. 각 청크에 메타데이터 추가 (ID, 길이, 조항 정보 등)
        2. 청크 크기가 너무 큰 경우 자동으로 제한
        3. 배치 단위로 임베딩 생성하여 토큰 한도 문제 방지
        4. FAISS merge 기능을 활용하여 배치별 결과 통합
    """
    try:
        print(f"   총 {len(text_chunks)}개 청크를 {batch_size}개씩 배치 처리...")
        
        # Document 객체들을 저장할 리스트 초기화
        documents = []
        
        # 각 텍스트 청크를 Document 객체로 변환하면서 메타데이터 추가
        for i, chunk in enumerate(text_chunks):
            # 청크 크기가 너무 큰 경우 제한 (토큰 한도 방지)
            if len(chunk) > 2000:
                chunk = chunk[:2000] + "..."
                print(f"   경고: 청크 {i}가 너무 커서 2000자로 제한했습니다.")
            
            # 각 청크에 추가할 메타데이터 구성
            metadata = {
                'chunk_id': i,  # 청크 고유 번호
                'chunk_length': len(chunk),  # 청크 길이
                'chunk_type': 'legal_document'  # 문서 유형
            }
            
            # 청크 내용에서 조항 정보 자동 추출
            # "제n조" 패턴을 찾아 메타데이터에 추가
            if '제' in chunk and '조' in chunk:
                article_match = re.search(r'제(\d+)조', chunk)
                if article_match:
                    metadata['article'] = f"제{article_match.group(1)}조"
            
            # Document 객체 생성하여 리스트에 추가
            documents.append(Document(page_content=chunk, metadata=metadata))
        
        # 배치별 벡터 저장소 생성 및 병합 과정
        vector_store = None
        total_batches = (len(documents) + batch_size - 1) // batch_size
        
        # 문서들을 배치 크기만큼 나누어 처리
        for i in range(0, len(documents), batch_size):
            batch_docs = documents[i:i + batch_size]
            batch_num = (i // batch_size) + 1
            
            print(f"   배치 {batch_num}/{total_batches} 처리 중... ({len(batch_docs)}개 문서)")
            
            # 첫 번째 배치인 경우 새로운 벡터 저장소 생성
            if vector_store is None:
                vector_store = FAISS.from_documents(batch_docs, embedding_model)
            else:
                # 이후 배치들은 기존 벡터 저장소에 병합
                batch_vector_store = FAISS.from_documents(batch_docs, embedding_model)
                vector_store.merge_from(batch_vector_store)
        
        print("   모든 배치 처리 완료")
        return vector_store, documents
        
    except Exception as e:
        raise RuntimeError(f"벡터 저장소 생성 실패: {str(e)}")

# 6. 키워드 기반 검색 함수
def keyword_search(query, documents, k=5):
    """
    단순한 키워드 매칭을 통한 문서 검색
    벡터 검색과 상호 보완적으로 사용하여 검색 정확도 향상
    
    Args:
        query (str): 검색 질의
        documents (list): Document 객체들의 리스트
        k (int): 반환할 상위 문서 수
        
    Returns:
        list: 관련도 순으로 정렬된 Document 객체들
        
    검색 로직:
        1. 질의와 문서의 단어 교집합 계산
        2. 교집합 크기를 질의 단어 수로 나누어 정규화
        3. 정확한 구문 매칭 시 보너스 점수 부여
        4. 점수 기준으로 상위 k개 문서 반환
    """
    # 질의를 소문자로 변환하고 단어 단위로 분할
    query_words = set(query.lower().split())
    
    # 각 문서의 점수를 계산할 리스트
    scores = []
    
    for i, doc in enumerate(documents):
        # 문서 내용을 소문자로 변환하고 단어 단위로 분할
        content_words = set(doc.page_content.lower().split())
        
        # 질의 단어와 문서 단어의 교집합 계산
        intersection = query_words.intersection(content_words)
        
        # 기본 점수: 교집합 크기를 질의 단어 수로 나누어 정규화 (0~1 범위)
        score = len(intersection) / len(query_words) if query_words else 0
        
        # 보너스 점수: 질의 전체가 문서에 정확히 포함된 경우
        if query.lower() in doc.page_content.lower():
            score += 0.5
        
        # (점수, 인덱스, 문서) 튜플로 저장
        scores.append((score, i, doc))
    
    # 점수 기준으로 내림차순 정렬하여 상위 k개 반환
    scores.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, _, doc in scores[:k]]

# 7. 하이브리드 검색 함수 (벡터 + 키워드 검색 결합)
def hybrid_search(query, vector_store, documents, k=5, alpha=0.7):
    """
    벡터 유사도 검색과 키워드 검색을 결합한 하이브리드 검색
    두 검색 방법의 장점을 결합하여 더 정확한 검색 결과 제공
    
    Args:
        query (str): 검색 질의
        vector_store: FAISS 벡터 저장소
        documents (list): Document 객체들의 리스트
        k (int): 최종 반환할 문서 수
        alpha (float): 벡터 검색 가중치 (0~1, 높을수록 벡터 검색 중시)
        
    Returns:
        list: 종합 점수로 정렬된 상위 k개 Document 객체들
        
    검색 과정:
        1. 벡터 유사도 검색으로 의미적으로 관련된 문서 찾기
        2. 키워드 검색으로 정확한 용어 매칭 문서 찾기
        3. 두 결과를 alpha 가중치로 결합
        4. 중복 문서 처리 및 최종 점수 계산
        5. 점수 순으로 정렬하여 상위 k개 반환
    """
    
    # 1. 벡터 유사도 검색 수행
    # 더 많은 후보를 가져와서 다양성 확보 (k*2개)
    vector_results = vector_store.similarity_search(query, k=k*2)
    
    # 2. 키워드 기반 검색 수행
    keyword_results = keyword_search(query, documents, k=k*2)
    
    # 3. 두 검색 결과를 점수와 함께 통합
    combined_results = {}
    
    # 벡터 검색 결과에 점수 부여 (alpha 가중치 적용)
    for i, doc in enumerate(vector_results):
        # 문서 내용을 고유 키로 사용
        doc_id = doc.page_content
        # 순위가 높을수록 높은 점수 (1.0에서 시작하여 순위에 따라 감소)
        vector_score = alpha * (1.0 - i / len(vector_results))
        
        combined_results[doc_id] = {
            'document': doc,
            'score': vector_score,
            'vector_rank': i + 1,
            'keyword_rank': None
        }
    
    # 키워드 검색 결과에 점수 부여 ((1-alpha) 가중치 적용)
    for i, doc in enumerate(keyword_results):
        doc_id = doc.page_content
        keyword_score = (1 - alpha) * (1.0 - i / len(keyword_results))
        
        if doc_id in combined_results:
            # 이미 벡터 검색에서 찾은 문서인 경우 점수 합산
            combined_results[doc_id]['score'] += keyword_score
            combined_results[doc_id]['keyword_rank'] = i + 1
        else:
            # 키워드 검색에서만 찾은 새로운 문서인 경우 추가
            combined_results[doc_id] = {
                'document': doc,
                'score': keyword_score,
                'vector_rank': None,
                'keyword_rank': i + 1
            }
    
    # 4. 종합 점수 기준으로 정렬하여 상위 k개 반환
    sorted_results = sorted(combined_results.values(), key=lambda x: x['score'], reverse=True)
    return [result['document'] for result in sorted_results[:k]]

# 8. 한국어 법률 문서 전용 프롬프트 생성 함수
def create_korean_legal_prompt():
    """
    한국어 법률 문서 특성에 맞춘 전용 프롬프트 템플릿 생성
    
    Returns:
        PromptTemplate: 법률 문서 질의응답을 위한 프롬프트 템플릿
        
    프롬프트 특징:
        - 법조문 인용의 정확성 강조
        - 전문 용어에 대한 쉬운 설명 요구
        - 조항 간 연관성 설명 포함
        - 실무적 적용 방법 제시
        - 불확실한 내용에 대한 명시적 언급
    """
    template = """당신은 한국 세법 전문가입니다. 주어진 법률 문서를 바탕으로 정확하고 자세한 답변을 제공해야 합니다.

다음 규칙을 반드시 따르세요:
1. 법조문의 조항, 항, 호, 목을 정확히 인용하세요
2. 전문 용어를 사용할 때는 쉬운 설명을 함께 제공하세요
3. 관련 조항들 간의 연관성을 설명하세요
4. 실무적 적용 방법도 함께 설명하세요
5. 불확실한 내용이 있으면 명시적으로 언급하세요

참고 문서:
{context}

질문: {question}

위 법률 문서를 바탕으로 정확하고 자세한 답변을 제공해주세요. 관련 조항을 인용하며 설명해주세요."""

    return PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

# 9. LLM을 활용한 질문 응답 함수
def query_with_llm(query, vector_store, documents):
    """
    하이브리드 검색과 고성능 LLM을 결합한 질문 응답 시스템
    
    Args:
        query (str): 사용자 질문
        vector_store: FAISS 벡터 저장소
        documents (list): Document 객체들의 리스트
        
    Returns:
        dict: 답변, 참고 문서, 사용된 컨텍스트를 포함한 응답 딕셔너리
        
    처리 과정:
        1. GPT-4o-mini 모델로 LLM 초기화 (높은 정확도)
        2. 하이브리드 검색으로 관련 문서 7개 검색
        3. 검색된 문서들을 하나의 컨텍스트로 결합
        4. 법률 문서 전용 프롬프트 적용
        5. LLM으로 최종 답변 생성
    """
    try:
        # LLM 모델 설정
        llm = ChatUpstage(
            model="solar-pro",
            base_url="https://api.upstage.ai/v1",
            temperature=0.5
        )
        
        print(f"질의: {query}")
        print("하이브리드 검색 수행 중...")
        
        # 하이브리드 검색으로 관련 문서 검색
        # k=7로 설정하여 충분한 컨텍스트 확보
        # alpha=0.7로 설정하여 벡터 검색을 더 중시 (의미적 유사도 우선)
        relevant_docs = hybrid_search(query, vector_store, documents, k=7, alpha=0.7)
        
        print(f"검색된 관련 문서: {len(relevant_docs)}개")
        
        # 검색된 문서들을 하나의 컨텍스트로 결합
        # 각 문서에 번호를 매겨 구분하기 쉽게 구성
        context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(relevant_docs)])
        
        # 한국어 법률 문서에 특화된 프롬프트 사용
        prompt = create_korean_legal_prompt()
        
        # 최종 프롬프트 생성 (컨텍스트와 질문 삽입)
        formatted_prompt = prompt.format(context=context, question=query)
        
        print("LLM 응답 생성 중...")
        
        # LLM에 프롬프트 전달하여 답변 생성
        response = llm.invoke(formatted_prompt)
        
        # 결과를 딕셔너리 형태로 반환
        return {
            "answer": response.content,
            "source_documents": relevant_docs,
            "context_used": context
        }
        
    except Exception as e:
        raise RuntimeError(f"LLM 응답 생성 실패: {str(e)}")

# 10. 검색 품질 평가 함수
def evaluate_context_quality(query, retrieved_docs):
    """
    검색된 문서들의 품질을 정량적으로 평가
    
    Args:
        query (str): 원본 질의
        retrieved_docs (list): 검색된 Document 객체들
        
    Returns:
        float: 0~1 범위의 품질 점수 (1에 가까울수록 높은 품질)
        
    평가 기준:
        1. 키워드 매칭률 (70% 가중치): 질의 단어가 문서에 포함된 비율
        2. 문서 길이 적절성 (30% 가중치): 너무 짧거나 길지 않은 적절한 길이
    """
    # 질의를 단어 단위로 분할하고 소문자 변환
    query_words = set(query.lower().split())
    
    # 각 문서별 품질 점수 계산
    quality_scores = []
    for doc in retrieved_docs:
        # 문서 내용을 단어 단위로 분할하고 소문자 변환
        doc_words = set(doc.page_content.lower().split())
        
        # 키워드 매칭 점수 계산
        # 질의 단어 중 문서에 포함된 단어의 비율
        keyword_match = len(query_words.intersection(doc_words)) / len(query_words)
        
        # 문서 길이 점수 계산
        # 1000자를 기준으로 정규화 (1000자 이상이면 1.0점)
        length_score = min(len(doc.page_content) / 1000, 1.0)
        
        # 종합 점수 계산 (키워드 매칭 70% + 길이 적절성 30%)
        total_score = (keyword_match * 0.7) + (length_score * 0.3)
        quality_scores.append(total_score)
    
    # 전체 문서의 평균 품질 점수 반환
    avg_quality = sum(quality_scores) / len(quality_scores) if quality_scores else 0
    return avg_quality


In [33]:
from langchain_openai import OpenAIEmbeddings

# 메인 실행 부분
if __name__ == "__main__":
    # 처리할 DOCX 파일 경로 설정
    docx_path = "/Users/eon/skRookies/myLangChain/mylangchain-app/src/mylangchain_app/data/tax_with_table.docx"
    
    print("개선된 RAG 파이프라인 실행")
    print("=" * 50)
    
    # 1단계: 문서 로드
    print("1. 문서 로드 중...")
    text = load_docx_advanced(docx_path)
    print(f"   문서 로드 완료: {len(text):,} 문자")
    
    # 2단계: 문서 분할
    print("\n2. 문서 분할 중...")
    text_chunks = advanced_split_text(text, chunk_size=600, chunk_overlap=100)
    print(f"   문서 분할 완료: {len(text_chunks)}개 청크 생성")
    
    # 청크 크기 통계 분석 및 출력
    chunk_lengths = [len(chunk) for chunk in text_chunks]
    avg_length = sum(chunk_lengths) / len(chunk_lengths)
    max_length = max(chunk_lengths)
    print(f"   평균 청크 길이: {avg_length:.0f}자, 최대 길이: {max_length}자")
    
    # 3단계: 임베딩 모델 초기화
    print("\n3. 임베딩 모델 초기화...")
    
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-small",
    )
    print("   임베딩 모델 초기화 완료")
    
    # 4단계: 벡터 저장소 생성
    print("\n4. 벡터 저장소 생성 중...")
    # 배치 크기 30으로 설정하여 토큰 한도 문제 방지
    vector_store, documents = create_vector_store(text_chunks, embedding_model, batch_size=30)
    print("   벡터 저장소 생성 완료")
    
    # 5단계: 질의 실행
    print("\n5. 질의 실행 중...")
   #query = "총수입금액 불산입에 대하여 설명해 주세요."
    query = "비과세소득의 종류에 대하여 설명해 주세요."
    results = query_with_llm(query, vector_store, documents)
    
    # 6단계: 검색 품질 평가
    context_quality = evaluate_context_quality(query, results["source_documents"])
    
    # 7단계: 결과 출력
    print("\n" + "=" * 60)
    print("AI의 답변:")
    print("=" * 60)
    print(results["answer"])
    
    print("\n" + "=" * 60)
    print("검색 결과 요약:")
    print("=" * 60)
    print(f"참고한 문서 조각 수: {len(results['source_documents'])}개")
    print(f"컨텍스트 품질 점수: {context_quality:.2f}/1.00")
    print(f"총 컨텍스트 길이: {len(results['context_used']):,} 문자")
    
    print("\n" + "=" * 60)
    print("참고한 문서 미리보기:")
    print("=" * 60)
    # 상위 3개 문서의 일부만 미리보기로 출력
    for i, doc in enumerate(results["source_documents"][:3]):
        # 200자까지만 미리보기로 표시
        preview = doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content
        print(f"\n[문서 {i+1}] {preview}")
            # 5단계: 질의 실행
    print("=" * 60)
    print("\n5. 질의 실행 중...")
    query = "총수입금액 불산입에 대하여 설명해 주세요."
    results = query_with_llm(query, vector_store, documents)
    
    # 6단계: 검색 품질 평가
    context_quality = evaluate_context_quality(query, results["source_documents"])
    
    # 7단계: 결과 출력
    print("\n" + "=" * 60)
    print("AI의 답변:")
    print("=" * 60)
    print(results["answer"])
    
    print("\n" + "=" * 60)
    print("검색 결과 요약:")
    print("=" * 60)
    print(f"참고한 문서 조각 수: {len(results['source_documents'])}개")
    print(f"컨텍스트 품질 점수: {context_quality:.2f}/1.00")
    print(f"총 컨텍스트 길이: {len(results['context_used']):,} 문자")
    
    print("\n" + "=" * 60)
    print("참고한 문서 미리보기:")
    print("=" * 60)
    # 상위 3개 문서의 일부만 미리보기로 출력
    for i, doc in enumerate(results["source_documents"][:3]):
        # 200자까지만 미리보기로 표시
        preview = doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content
        print(f"\n[문서 {i+1}] {preview}")
    print("=" * 60)
    

개선된 RAG 파이프라인 실행
1. 문서 로드 중...
   문서 로드 완료: 289,214 문자

2. 문서 분할 중...
   문서 분할 완료: 662개 청크 생성
   평균 청크 길이: 472자, 최대 길이: 600자

3. 임베딩 모델 초기화...
   임베딩 모델 초기화 완료

4. 벡터 저장소 생성 중...
   총 662개 청크를 30개씩 배치 처리...
   배치 1/23 처리 중... (30개 문서)
   배치 2/23 처리 중... (30개 문서)
   배치 3/23 처리 중... (30개 문서)
   배치 4/23 처리 중... (30개 문서)
   배치 5/23 처리 중... (30개 문서)
   배치 6/23 처리 중... (30개 문서)
   배치 7/23 처리 중... (30개 문서)
   배치 8/23 처리 중... (30개 문서)
   배치 9/23 처리 중... (30개 문서)
   배치 10/23 처리 중... (30개 문서)
   배치 11/23 처리 중... (30개 문서)
   배치 12/23 처리 중... (30개 문서)
   배치 13/23 처리 중... (30개 문서)
   배치 14/23 처리 중... (30개 문서)
   배치 15/23 처리 중... (30개 문서)
   배치 16/23 처리 중... (30개 문서)
   배치 17/23 처리 중... (30개 문서)
   배치 18/23 처리 중... (30개 문서)
   배치 19/23 처리 중... (30개 문서)
   배치 20/23 처리 중... (30개 문서)
   배치 21/23 처리 중... (30개 문서)
   배치 22/23 처리 중... (30개 문서)
   배치 23/23 처리 중... (2개 문서)
   모든 배치 처리 완료
   벡터 저장소 생성 완료

5. 질의 실행 중...
질의: 비과세소득의 종류에 대하여 설명해 주세요.
하이브리드 검색 수행 중...
검색된 관련 문서: 7개
LLM 응답 생성 중...

AI의 답변:
### 비과세